<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_system_firebase_budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q firebase-admin

In [15]:
import json, re
import google.generativeai as genai
from google.colab import userdata
prompt = '''
請根據輸入的文字，自動分析並拆解成以下欄位資訊，並直接輸出成 json 字串：
收支（收入或支出）
類別（請從以下類別中選擇最適合的）
【支出類別】：餐飲, 超商, 雜貨, 交通, 生活日用品, 房租, 房貸, 水電瓦斯, 網路, 電信費, 娛樂, 教育, 學習, 醫療保健, 服飾美容, 禮品, 交際, 寵物, 家庭支出, 其他支出
【收入類別】：教課收入,定線收入,薪資收入, 獎金, 投資收入, 利息收入, 退稅, 補助, 二手交易, 紅包, 禮金, 其他收入
項目（簡單描述內容）
金額（數字）
備註（如果提到早中晚餐，可以把吃的東西放在項目，早中晚餐放在備註，或在哪裡買什麼，地點就可以放在備註，以此類推）
請以以下格式範例回覆，不要有其他文字或說明，前後也不要加任何東西

  "收支": "支出",
  "類別": "餐飲",
  "項目": "牛排",
  "金額": 128,
  "備註": "老婆付"

如果一次有好幾個品項跟金額，品項就用半形逗號隔開放一起，金額直接加總，全整合成一筆
'''


def get_dict(data):
    match = re.search(r'({.*})', data, re.DOTALL)
    json_string = match.group(1) if match else None
    return json.loads(json_string) if json_string else None


genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")

def generate_response(info):
    response = model.generate_content(info + prompt)
    return get_dict(response.text)



In [4]:
from google.colab import userdata

import firebase_admin
from firebase_admin import credentials, firestore
import json

service_account_info = json.loads(userdata.get('FIREBASE_SERVICE_ACCOUNT_JSON'))
cred = credentials.Certificate(service_account_info)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [20]:
import datetime
def save_to_firestore(data):
  try:
    data['timestamp'] = datetime.datetime.now()
    db.collection('expenses').add(data)
    print("Data saved to Firestore successfully!")
  except json.JSONDecodeError:
      print("Invalid JSON format. Please check the output from generate_response().")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage
text_input = input('輸入項目：')
dict_output = generate_response(text_input)
print(dict_output)
save_to_firestore(dict_output)

輸入項目：演講費3000
{'收支': '收入', '類別': '教課收入', '項目': '演講', '金額': 3000, '備註': ''}
Data saved to Firestore successfully!


In [21]:
# prompt: 從firestore中找出支出的項目

import firebase_admin
from firebase_admin import credentials, firestore

# ... (Your existing code)

def get_expenses_from_firestore():
    """Retrieves all expense documents from the 'expenses' collection."""
    try:
        expenses_ref = db.collection('expenses')
        expenses = expenses_ref.stream()
        expenses_list = []  # list to store expenses

        for expense in expenses:
            expense_data = expense.to_dict()
            expenses_list.append(expense_data)
            #print(f'{expense.id} => {expense.to_dict()}')

        return expenses_list
    except Exception as e:
        print(f"An error occurred while retrieving expenses: {e}")
        return None

# Example usage:
retrieved_expenses = get_expenses_from_firestore()

if retrieved_expenses:
    print(retrieved_expenses) # process your expenses data here
    # Example: Iterate through the list of expenses
    #for expense in retrieved_expenses:
    #    print(expense)


[{'項目': '牛奶', '金額': 800, '類別': '雜貨', '收支': '支出', '備註': '好市多'}, {'項目': '演講', '金額': 3000, '類別': '教課收入', 'timestamp': DatetimeWithNanoseconds(2025, 4, 28, 13, 39, 39, 657142, tzinfo=datetime.timezone.utc), '收支': '收入', '備註': ''}, {'項目': '西堤', '金額': 1024, '類別': '餐飲', '收支': '支出', '備註': '晚餐'}]
